In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings

In [2]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [10]:
question = "what is the result of 1 plus 2?"

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class mathemtician. output the result only"),
    ("user", "{input}")
])

In [16]:
chain = prompt | llm | StrOutputParser()

In [17]:
chain.invoke({"input": question})

'3'

In [19]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [21]:
embeddings = OpenAIEmbeddings()

In [22]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [24]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [26]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [29]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [30]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing capabilities for prototyping, debugging, creating test sets, comparison views, playground environments for rapid iteration and experimentation, beta testing, capturing feedback, annotating traces, adding runs to datasets, production monitoring, A/B testing, automations, and threads view for multi-turn applications. These features allow developers to efficiently test and evaluate their LLM applications at various stages of development.


In [31]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [32]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing ‚Äî a

In [36]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

In [37]:
import os
os.environ["TAVILY_API_KEY"] = "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"

In [38]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [39]:
search("hello")

/Users/wangqiang/miniforge3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'url': 'https://tech-gate.org/2024/03/24/non-hot-industries-can-enchantment-to-traders-method-too/',
  'content': "Δ Don't Miss Tech large Fujitsu says it was hacked, warns of information breach by bourbiza mohamed March 18, 2024, 12:41 pm How Clear Vitality Tax Breaks May Gasoline a US Wooden Burning Growth by bourbiza mohamed March 15, 2024, 9:24 am Politics divide state-level clear power targets; renewable capability rising by bourbiza mohamed November 22, 2023, 6:16 pm Singapore’s forthcoming LNG terminal faces no significant stranding possibility from renewables push: minister | News | Eco-Business by bourbiza mohamed March 2, 2024, 5:40 am Quickly after AWS and Google, Microsoft states it really is eradicating Azure ‘egress’ particulars switch prices — however with caveats by bourbiza mohamed March 14, 2024, 6:14 pm Sports activities And Web3 Collides Throughout Sporting Crypto Social NYC Version by bourbiza mohamed December 1, 2023, 11:06 pm AI is ‘rubbish’ however continues t

In [40]:
tools = [retriever_tool, search]

In [41]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [42]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})




> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for ü¶úÔ∏èüõ†Ô∏è LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strat

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in various ways throughout the application development lifecycle. Here are some key ways LangSmith can assist with testing:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategies, and other parameters for LLM applications.\n\n2. Debugging: LangSmith provides tracing capabilities to help developers understand how the model is performing and debug issues when things go wrong.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications using LangSmith.\n\n4. Comparison View: LangSmith offers a comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application.\n\n5. Playground: LangSmith provides a playground environment for rapid iteration and experimentation with different prompts and models.\n\n6. Beta Testing: Developers ca